In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-134604")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EKAJSHZT8 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-134604
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-134604


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name="test-prj1"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os
from azureml.core import Environment
from azureml.core import ScriptRunConfig

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.5,1.5),
        '--max_iter': choice(range(50,200))
    }
)
sklearn_env=Environment.get(workspace=ws, name='AzureML-Tutorial')
# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor= 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
compute_target = ws.compute_targets[cpu_cluster_name]
est = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      arguments=['--C', 1, '--max_iter', 100],
                      compute_target=compute_target,
                      environment=sklearn_env)
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
hyperparameter_sampling=ps,
policy=policy,
primary_metric_name="Accuracy",
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=10,
max_concurrent_runs=4
)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
experiment = Experiment(ws, "udacity-project")
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:',parameter_values[5])
print('\n max-iter:',parameter_values[3])

best_run.download_file(
    best_run.get_file_names()[-1],
    output_file_path='outputs'
)
best_hyperdrive_model = best_run.register_model(
    model_name="best_hyperdrive_model",
    model_path='outputs/best_hyperdrive_model.joblib',
   
)

Best Run Id:  HD_250343d2-d2ca-4983-9d95-0c30dfde5261_0

 Accuracy: 0.9162367223065251

 C: 1.0106777993341345

 max-iter: 100


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
path= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path=path)

In [14]:
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.core.run import Run
from azureml.core import Dataset
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
df=x.join(y)
os.makedirs('data',exist_ok=True)
local_path='data/prepared_data.csv'
df.to_csv(local_path)
workspace=Workspace(ws.subscription_id,ws.resource_group,ws._workspace_name)
datastore=workspace.get_default_datastore()
datastore.upload(src_dir='data',target_path='data')
ds=Dataset.Tabular.from_delimited_files(datastore.path(local_path))
training, validation=ds.train_test_split(percentage=0.8, seed=23)

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_setting= {
    "experiment_timeout_minutes": 30,
    "n_cross_validations": 5,
    "primary_metric": "accuracy",
    "max_concurrent_iterations": 4}
automl_config = AutoMLConfig(
    task="classification",
    training_data=training,
    label_column_name='y',
    compute_target=compute_target,
    **automl_setting
)

In [13]:
# Submit your automl run
experiment=Experiment(ws,'automl')
automl_run=experiment.submit(automl_config,show_output=True)

Running on remote.
No run_configuration provided, running on test-prj1 with default configuration
Running on remote compute: test-prj1
Parent Run ID: AutoML_a1533c27-2267-4200-8e17-24f55f42b436

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one c

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###